## Распознование текста из микрофона

In [ ]:
sudo apt-get install portaudio19-dev

In [1]:
!environment/bin/pip install SpeechRecognition
!environment/bin/pip install PyAudio

In [2]:
import speech_recognition as sr

In [3]:
recognizer = sr.Recognizer()

In [4]:
recognizer.listen??

In [4]:
with sr.Microphone() as micro:
    print(1)
    audio = recognizer.listen(micro)
    print(2)

1
2


In [54]:
with open('data/microphone-results.wav', "wb") as f:
    f.write(audio.get_wav_data())

In [5]:
recognizer.recognize_google(audio, language='en-EN')

'speak speak speak speak speak speak'

## Распознование текста из аудиофайла

In [31]:
recognizer = sr.Recognizer()

In [58]:
with sr.AudioFile('data/microphone-results.wav') as source:
    audio1 = recognizer.record(source)

In [59]:
recognizer.recognize_google(audio1, language='en-EN')

'this is a sound detection chick 111 detection check'

## Text topic extraction

In [36]:
!py3_env/bin/pip install spacy
!py3_env/bin/pip install pymorphy2

In [37]:
import spacy

In [39]:
# spacy.load('ru')

In [60]:
from spacy.lang.en import English

In [61]:
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [63]:
text = recognizer.recognize_google(audio, language='en-EN')
tokenize(text)

['this', 'is', 'a', 'sound', 'detection', 'chick', '111', 'detection', 'check']

In [3]:
!environment/bin/pip install nltk

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [16]:
get_lemma("их")

'их'

In [78]:
for token in tokenize(text):
    print(get_lemma(token), get_lemma2(token))

this this
be is
a a
sound sound
detection detection
chick chick
111 111
detection detection
check check


In [11]:
nltk.download('stopwords')
# en_stop = set(nltk.corpus.stopwords.words('english'))
en_stop = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
list(en_stop)[:5]

['со', 'при', 'чтобы', 'может', 'без']

In [90]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    return [get_lemma(token) for token in tokens 
            if len(token) > 4 and token not in en_stop]

In [91]:
text_data = [prepare_text_for_lda(text)]
text_data

[['sound', 'detection', 'chick', 'detection', 'check']]

In [82]:
!py3_env/bin/pip install gensim

In [86]:
from gensim import corpora

dictionary = corpora.Dictionary(text_data)
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in text_data]
print(corpus)

Dictionary(4 unique tokens: ['check', 'chick', 'detection', 'sound'])
[[(0, 1), (1, 1), (2, 2), (3, 1)]]


In [89]:
dictionary.doc2bow?? # document to bag of words

In [88]:
import gensim

NUM_TOPICS = 2
NUM_WORDS = 3

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.251*"detection" + 0.250*"check" + 0.250*"sound"')
(1, '0.359*"detection" + 0.214*"chick" + 0.214*"sound"')


## загрузка аудиофайла из youtube

In [31]:
!py3_env/bin/pip install youtube_dl
!py3_env/bin/pip install oauth2client
!py3_env/bin/pip install pydub

In [14]:
import youtube_dl

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192'
    }],
    'outtmpl': '1',
    'prefer_ffmpeg': True,
    'keepvideo': False
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=J5O5iRpLXKY&t=637s'])

[youtube] J5O5iRpLXKY: Downloading webpage
[youtube] J5O5iRpLXKY: Downloading video info webpage
[download] Destination: 1
[download] 100% of 15.11MiB in 00:13.00KiB/s ETA 00:000
[ffmpeg] Destination: wav
Deleting original file 1 (pass -k to keep)


In [15]:
from pydub import AudioSegment
t1 = 110 * 1000 #Works in milliseconds
t2 = 200 * 1000
newAudio = AudioSegment.from_wav("data/1.wav")
newAudio = newAudio[t1:t2]
newAudio.export('data/3.wav', format="wav")

<_io.BufferedRandom name='data/3.wav'>

In [57]:
recognizer = sr.Recognizer()
with sr.AudioFile('data/3.wav') as source:
    audio1 = recognizer.record(source)
recognizer.recognize_google(audio1, language='ru_RU')

'чтобы после очень крутой 18 года попробовать себя на западе А это произошло наверное летом летом 18 летом 18 года Так как только артист увидел офсет увидел эту ссылку Он резко отправил своим продюсером и сказал у меня переписку показывали потом эти же продюсер искал это моё секретное оружие Давайте звать парня и потихоньку очень медленно начинался запускается процесс То есть как бы это ни было моментально там о Попов и мы уже Я уже тут В начале обсуждал Ну поскольку у них стартовал тур совместно с дрейком не улавливает of цитата 11 чуваков из группы migos когда архитектуре это всегда любой клип запись альбома идёт очень медленно потому что они ежедневно просто путешествую фотосессии концерты в каждом городе по два по три концерта все солдаут были и наш тоже клип он сильно двигался медленно но ближе наверное в октябре уже нам Мне позвонили сказали всё Поехали Давай всё снимать придумывай идею У артиста из Кто такие заметную столько-то заметок что он хочет видеть и мы сейчас я с этим св

## Analyse podcast topics with wor2vec

In [11]:
import nltk
from gensim.models import Word2Vec

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [60]:
with open('data/podcast.txt', 'r+') as f:
    text = f.read()
text[100:200]

'й как нервничал тогда секс это правда что вы гений с юрий будет дуть дуть будет юрий мы живем в очен'

In [61]:
tokens = text.split()
print(len(tokens))
en_stop = set(nltk.corpus.stopwords.words('russian'))
filtered_tokens = [token for token in tokens if token not in en_stop]
print(len(filtered_tokens))

6100
3664


In [62]:
model = Word2Vec([filtered_tokens], size=50, window=5, min_count=3, workers=4)

In [63]:
model.wv.vocab

{'фсб': <gensim.models.keyedvectors.Vocab at 0x7f14dcf6b898>,
 'ещё': <gensim.models.keyedvectors.Vocab at 0x7f14dcf6b080>,
 'это': <gensim.models.keyedvectors.Vocab at 0x7f14dcbe90f0>,
 'правда': <gensim.models.keyedvectors.Vocab at 0x7f14dcbe9438>,
 'юрий': <gensim.models.keyedvectors.Vocab at 0x7f14dcbe9240>,
 'дуть': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb3e80>,
 'очень': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb33c8>,
 'время': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb3b70>,
 'любой': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb36a0>,
 'человека': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb34a8>,
 'украины': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb3320>,
 'россии': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb34e0>,
 'поэтому': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb3a90>,
 'хочу': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb30f0>,
 'важно': <gensim.models.keyedvectors.Vocab at 0x7f14dcbb32e8>,
 'люди': <gensim.models.keyedvectors.Voc

In [64]:
word_vectors = [model.wv[token] for token in filtered_tokens if token in model.wv.vocab]
word_vectors[:5]

[array([ 1.0114988e-02,  1.0283422e-02, -7.6029478e-03,  7.6749474e-03,
        -1.1138117e-02,  8.4467074e-03, -8.1775896e-03, -2.3438225e-03,
        -6.2006842e-03, -5.1858430e-03,  9.5886188e-03,  6.7679037e-04,
        -4.7054570e-03, -6.5293941e-03,  7.8719360e-04, -5.5366778e-03,
         2.6090550e-03,  9.4614988e-03, -4.9308022e-03,  8.9907041e-03,
         5.9430534e-03,  1.2019658e-03, -6.3871947e-04,  8.0914861e-03,
         8.1921397e-03, -7.5835893e-03,  8.6571760e-03,  7.1957642e-03,
         2.8422438e-03, -4.3068084e-04,  7.4822013e-04, -8.6634504e-03,
        -4.0765391e-03,  1.5294854e-05, -1.4949703e-03,  3.9984584e-03,
         8.3919642e-03, -5.7147550e-03, -4.4320505e-03, -7.6206676e-03,
         3.8003556e-03, -1.1389580e-02, -5.9444685e-03,  4.6900404e-03,
        -2.3879665e-03,  8.1651146e-03, -9.1546541e-03,  3.4984582e-04,
         9.2595657e-03, -6.1629759e-03], dtype=float32),
 array([ 0.01021524,  0.00344352,  0.00098531, -0.00333985,  0.00463316,
      

In [74]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(word_vectors)

In [75]:
from collections import Counter
clusters_counter = Counter(kmeans.predict(word_vectors))
clusters_counter

Counter({19: 56,
         11: 22,
         3: 123,
         12: 56,
         2: 166,
         0: 80,
         8: 47,
         10: 105,
         16: 84,
         7: 51,
         9: 35,
         18: 169,
         14: 52,
         17: 30,
         1: 30,
         6: 163,
         13: 76,
         4: 121,
         5: 44,
         15: 34})

In [81]:
for i, cluster in enumerate(sorted(clusters_counter, 
                                   key=lambda cluster: clusters_counter[cluster], reverse=True)):
    cluster_vector = kmeans.cluster_centers_[cluster]
    topic = model.wv.similar_by_vector(cluster_vector, topn=5, restrict_vocab=None)
    print('TOPIC_' + str(i+1), *[word[0] for word in topic])

TOPIC_1 сказал история которого поэтому многие
TOPIC_2 юрий россия моё чемпионата украине
TOPIC_3 мира говорить ельцин этому слова
TOPIC_4 это вещи говорить россию имею
TOPIC_5 который сказать вообще вопрос украинцы
TOPIC_6 говорю понимаю человека других дома
TOPIC_7 знаю крыма считаю значит фифа
TOPIC_8 человек происходит думаю дуть стран
TOPIC_9 федерации фамилии людей путина власть
TOPIC_10 чемпионат фсб этих программа сказали
TOPIC_11 крым несколько страны украинский знаем
TOPIC_12 году хочу руководители 500 причинам
TOPIC_13 украины говорить украина могу правильно
TOPIC_14 очень российской второе встал долларов
TOPIC_15 всё власти действительно это спецслужб
TOPIC_16 россии руководил первое считаю невозможно
TOPIC_17 почему знаете россияне возможно лет
TOPIC_18 люди говорил белорусы чём считаю
TOPIC_19 которые первое отношусь сказал путина
TOPIC_20 президентом ещё знали спецслужб сказали
